### Pair Programming ETL Carga I

___

Es el momento de meter todos nuestros datos en SQL 💪🏽!!! En este ejercicio nos crearemos dos tablas en una BBDD creada por nosotras.
📌 Nota Todo lo tendremos que hacer desde jupyter notebook



In [1]:
import pandas as pd
import mysql.connector

1. Cread la BBDD con el nombre de energía. 

In [2]:
def crear_bbdd(nombre_bbdd):

    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="AlumnaAdalab",
      auth_plugin = 'mysql_native_password') 
    print("Conexión realizada con éxito")
    
    mycursor = mydb.cursor()

    try:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
        print(mycursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [3]:
crear_bbdd("energy")

Conexión realizada con éxito
MySQLCursor: CREATE DATABASE IF NOT EXISTS energy;


2. Cread las tablas de la BBDD:
- Tabla fechas
- Tabla nacional_renovable_no_renovable
- Tabla comunidades_renovable_no_renovable
- Tabla comunidades

In [4]:
def crear_insertar_tabla(nombre_bbdd, query):
    
    cnx = mysql.connector.connect(host="localhost", user="root", password="AlumnaAdalab", database=f"{nombre_bbdd}", auth_plugin = 'mysql_native_password') 

    mycursor = cnx.cursor()

    try: 
        mycursor.execute(query)
        cnx.commit() 

    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [5]:
crear_tablas = """ CREATE TABLE IF NOT EXISTS `energy`.`fechas` (
                `id_date` INT NOT NULL AUTO_INCREMENT,
                `date` DATE NOT NULL,
                PRIMARY KEY (`id_date`))
                ENGINE = InnoDB;
                
                CREATE TABLE IF NOT EXISTS `energy`.`comunidades` (
                `id_location` INT NOT NULL,
                `location` VARCHAR(45) NOT NULL,
                PRIMARY KEY (`id_location`))
                ENGINE = InnoDB;
                
                CREATE TABLE IF NOT EXISTS `energy`.`comunidades_renovable_no_renovable` (
                `id_comunidades_renovable_no_renovable` INT NOT NULL AUTO_INCREMENT,
                `value` FLOAT NULL,
                `percentage` FLOAT NULL,
                `energy_type` VARCHAR(45) NULL,
                `comunidades_id_location` INT NOT NULL,
                `fechas_id_date1` INT NOT NULL,
                PRIMARY KEY (`id_comunidades_renovable_no_renovable`),
                INDEX `fk_comunidades_renovable_no_renovable_comunidades1_idx` (`comunidades_id_location` ASC) VISIBLE,
                INDEX `fk_comunidades_renovable_no_renovable_fechas1_idx` (`fechas_id_date1` ASC) VISIBLE,
                CONSTRAINT `fk_comunidades_renovable_no_renovable_comunidades1`
                FOREIGN KEY (`comunidades_id_location`)
                REFERENCES `energy`.`comunidades` (`id_location`)
                ON DELETE NO ACTION
                ON UPDATE NO ACTION,
                CONSTRAINT `fk_comunidades_renovable_no_renovable_fechas1`
                FOREIGN KEY (`fechas_id_date1`)
                REFERENCES `energy`.`fechas` (`id_date`)
                ON DELETE NO ACTION
                ON UPDATE NO ACTION)
                ENGINE = InnoDB;
                
                CREATE TABLE IF NOT EXISTS `energy`.`nacional_renovable_no_renovable` (
                `idnacional_renovable_no_renovable` INT NOT NULL AUTO_INCREMENT,
                `value` FLOAT NULL,
                `percentage` FLOAT NULL,
                `energy_type` VARCHAR(45) NULL,
                `fechas_id_date` INT NOT NULL,
                PRIMARY KEY (`idnacional_renovable_no_renovable`),
                INDEX `fk_nacional_renovable_no_renovable_fechas1_idx` (`fechas_id_date` ASC) VISIBLE,
                CONSTRAINT `fk_nacional_renovable_no_renovable_fechas1`
                FOREIGN KEY (`fechas_id_date`)
                REFERENCES `energy`.`fechas` (`id_date`)
                ON DELETE NO ACTION
                ON UPDATE NO ACTION)
                ENGINE = InnoDB;"""

In [6]:
crear_insertar_tabla("energy", crear_tablas)

Use multi=True when executing multiple statements
Error Code: -1
SQLSTATE None
Message Use multi=True when executing multiple statements


Nos sale error en el commit pero, revisando Workbench, todas las tablas y foreign keys están correctamente creadas.


    
**BONUS** Insertar los datos en las tablas.
📌 Nota Esta parte del pair es optativa y no será considerada para la evaluación

In [7]:
df_energia = pd.read_csv("data/energias_años.csv", index_col=0)
df_ccaa = pd.read_csv("data/energias_comunidades.csv", index_col=0)

In [8]:
df_energia.head(2)

,value,percentage,energy_type,date
0,213821.42,0.36,Renovable,2020-01-01
1,270043.77,0.38,Renovable,2020-01-02


In [9]:
df_ccaa.head(2)

,value,percentage,energy_type,location,id_location,date
0,199198.10,1.00,No renovable,Ceuta,8744,2020-01-01
0,5602.26,0.03,Renovable,Melilla,8745,2020-01-01


Vamos a insertar en "comunidades"

In [10]:
cod_location = {"Ceuta": 8744, "Melilla": 8745, "Andalucía": 4, "Aragón": 5, "Cantabria": 6, "Castilla - La Mancha": 7, "Castilla y León": 8, "Cataluña": 9, "País Vasco": 10,
                        "Principado de Asturias": 11, "Comunidad de Madrid": 13, "Comunidad Foral de Navarra": 14, "Comunitat Valenciana": 15, "Extremadura": 16, "Galicia": 17,
                        "Illes Balears": 8743, "Canarias": 8742, "Región de Murcia": 21, "La Rioja": 20}

In [11]:
df_localidades = pd.DataFrame(pd.Series(cod_location)).reset_index()

In [12]:
df_localidades.head(2)

,index,0
0,Ceuta,8744
1,Melilla,8745


In [13]:
for indice, fila in df_localidades.iterrows():

    query_localidad = f"""
                INSERT INTO comunidades (id_location, location) 
                VALUES ( "{fila[0]}", "{fila["index"]}");"""
    
    crear_insertar_tabla("energy", query_localidad)

Vamos a insertar en "fechas"

In [14]:
fechas = pd.Series(df_energia["date"].unique())
fechas

0      2020-01-01
1      2020-01-02
2      2020-01-03
3      2020-01-04
4      2020-01-05
          ...    
726    2021-12-27
727    2021-12-28
728    2021-12-29
729    2021-12-30
730    2021-12-31
Length: 731, dtype: object

In [15]:
for fila in fechas:
    query_fechas = f"""
                INSERT INTO fechas (date)
                VALUES ('{fila}')"""
    
    crear_insertar_tabla("energy", query_fechas)

In [16]:
df_energia.head(1)

,value,percentage,energy_type,date
0,213821.42,0.36,Renovable,2020-01-01


In [17]:
df_ccaa.head(1)

,value,percentage,energy_type,location,id_location,date
0,199198.1,1.0,No renovable,Ceuta,8744,2020-01-01


In [18]:
def sacar_id_fecha(nombre_bbdd, fecha):
        
        cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                                    host='127.0.0.1', database=f"{nombre_bbdd}",  auth_plugin = 'mysql_native_password')
        mycursor = cnx.cursor()


        query_sacar_id_fecha = f"SELECT id_date FROM fechas WHERE date = '{fecha}'"
        
        try: 
            mycursor.execute(query_sacar_id_fecha)
            id_date = mycursor.fetchall()[0][0]
            return id_date
        
        except: 
            return "Sorry, no tenemos esa fecha en la BBDD y por lo tanto no te podemos dar su id. "

In [19]:
for indice, fila in df_energia.iterrows():
    
    id_fecha = sacar_id_fecha("energy", fila["date"])
    
    query_energia = f"""
                INSERT INTO nacional_renovable_no_renovable (value, percentage, energy_type, fechas_id_date) 
                VALUES ({fila["value"]}, {fila["percentage"]}, "{fila["energy_type"]}", {id_fecha});
                """
    crear_insertar_tabla("energy", query_energia)

In [22]:
for indice, fila in df_ccaa.iterrows():
    
    id_fecha = sacar_id_fecha("energy", fila["date"])
    
    query_energia = f"""
                INSERT INTO comunidades_renovable_no_renovable (value, percentage, energy_type, comunidades_id_location, fechas_id_date1) 
                VALUES ({fila["value"]}, {fila["percentage"]}, "{fila["energy_type"]}", "{fila["id_location"]}", {id_fecha});
                """
    crear_insertar_tabla("energy", query_energia)